# PRÁCTICA 1 BaseV1.1 - ALGORITMO BASADOS EN ENTORNOS Y TRAYECTORIAS
## La función movement_operator no es correcta y no hace falta matriz
El profesor me ha dicho que la forma de realizarlo es errónea y debería de la siguiente forma:

    soluciónActual
    numVecinos = 0b
    límiteVecinos = L
    For i = 1 to N
    For j = i to N
          If numVecinos < límiteVecinos
              numVecinos++
               Vecinos.add(vecino(I,j))
          Else
               MejorVeciono=MinCoste(Vecinos)
               If mejorVecino < soluciónActual
                    Soluciónactual= MejorVecino
                    Break;
                   

Entiendo que con el doble bucle for compruebas todos los vecinos posibles de una forma más sencilla. Pero hay algo que sigo sin comprender, no entiendo la existencia de la comparación 'numVecinos < limiteVecinos'. Entiendo que el propósito del algoritmo es seleccionar todos los vecinos candidatos y una vez hecho, comprobar cuál es el mejor. Pero de la forma en la que está escrito el pseudocódigo, nunca se llegaría al límite y tampoco se llegaría a comprobar cuál es el mejor vecino. Ya que el profesor comentó que era necesario explorar todos los vecinos (al menos en el primer apartado de la práctica, en el Greedy). Supongo que esta comparación será necesaria para tener un rango de exploración más grande o más pequeño en los siguientes apartados. Por lo tanto, debería agregar todos los vecinos y comprobar la mejor solución. Creo que lo mejor sería realizar la comprobación del mejor vecino y su comparación con la solución actual fuera del bucle.

## Código desarrollado en el archivo 'Practica1_Base'

In [17]:
import numpy as np
import pandas as pd


index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix

In [18]:
def movements_to_list(deltas_df):
    """
    Devuelve array con todos los movimientos realizados en las estaciones. Cada posicion del array esta compuesto por el id de la estacion y la cantidad de bicis retiradas/aniadidas en cada movimiento
    
    Parametros
        deltas_df : DataFrame
            Pandas DataFrame a convertir
    Return
        Numpy array
    """
    move_list = np.array([]) # Inicializamos el array
    rows = len(deltas_df.index) # Mediciones totales realizadas
    columns = len(deltas_df.columns) # Numero de estaciones
    
    # Recorreremos todas las mediciones e iremos transformando todas ellas que sean distintas de 0
    for i in range(0, rows):
        if i != 0:
            row = deltas_df.iloc[i] # Seleccionamos una medicion
            selec = row[row != 0] # Seleccionamos que estaciones han sufrido algun cambio
            for j in range(0, len(selec)):
                lst = np.array([[int(selec.index[j]), selec[j]]]) # Obtenemos un array con el index y el numero de bicis retiradas/aniadidas
                # En la setencia anterior, el primer corchete sirve para crear el array. Mientras que el segundo para crear una tupla tal que: [estacion, movements]
                if len(move_list) == 0:
                    move_list = lst # Asignamos la primera tupla como 
                else:
                    move_list = np.append(move_list, lst, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
    
    return move_list

### Función de evaluación

In [19]:
def evaluate(move_list, init_state, capacity, index_m, kms_m):
    '''
    La funcion estipula el numero de kilometros recorrido por los usuarios al no encontrar una bicicleta en su estación de preferencia y tener que desplazarse a la estaion mas cercana con bicicletas disponibles, o la distancia recorrida en caso de no haber sitio para dejar la bicicleta en la estación deseada y tener que desplazarse a la estacion mas cercana con capacidad. La distancia realizada a pie es 3 veces mayor que la recorrida en bicicleta
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de biciclestas alojadas en cada estacion al comienzo de la evaluacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
    Returns
        El numero total de kilometros recorridos por los usuarios
    '''
    tkms = 0 # Kilometros totales
    travel_kms = 0 # Kilometros recorridos por un usuario
    actual_state = init_state.copy() # Variable en la que iremos modificando el numero de biciclestas que se encuentran en cada estacion
    walk_multiplier = 3 # La distancia recorrida andando costara 3 veces mas que la recorrida en bici (valor calculado en kms_m es la distancia en bici)
    
    for move in move_list:
        station = move[0] # Guardamos la estacion 
        n_bicycle = actual_state[station] + move[1]  # Al numero de bicis de la estacion es cuestion, se le suma el numero de bicis desplazadas (este valor puede ser positivo o negativo) 
        
        # Tenemos que comprobar que el numero de desplazamientos es posible. Es decir, en caso de que se hayan retirado biciletas, que hubiese suficientes para suplir la demanda.
        # Y caso de que se quisiera dejar bicicletas, que existan suficientes slots/espaciones disponibles en la estacion
        if n_bicycle >= 0 and n_bicycle <= capacity[station]: 
            actual_state[station] = n_bicycle # Como el numero obtenido es posible, actualizamos el estado actual de la estacion
        else:
            # En caso de que no existan bicicletas suficientes para suplir la demanda en la estacion
            if n_bicycle < 0:
                actual_state[station] = 0 # La estacion ahora tiene 0 bicicletas
                search = abs(n_bicycle) # abs() nos devuelve el valor absoluto. Obtenemos el valor de bicicletas que necesitamos buscar
                nearest_station_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'bicycles') # Obtenemos la estacion mas cercana con capacidad suficiente de bicicletas
                
                # Calculamos los kms recorridos para llegar a la estacion mas cercana
                travel_kms = kms_m[station][nearest_station_index] * search * walk_multiplier # Kms hacia la estacion * numero de bicicletas a buscar * multiplicador por andar
                
                # Actualizamos el estado de la estacion mas cercana
                nearest_station = index_m[station][nearest_station_index] # Estacion mas cercana a la nuestra
                actual_state[nearest_station] -= search # Le restamos el numero de bicicletas necesario
            
            # En caso de que no existan slots suficientes para suplir la demanda en la estacion
            elif n_bicycle > capacity[station]:
                actual_state[station] = capacity[station] # Ahora la estacion esta llena
                search = n_bicycle - capacity[station] # Obtenemos el numero de slots a buscar
                nearest_station_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'slots') # Obtenemos la estacion mas cercana con capacidad suficiente de slots
                
                # Calculamos los kms recorridos para llegar a la estacion mas cercana
                travel_kms = kms_m[station][nearest_station_index] * search # Kms hacia la estacion * numero de slots a buscar
                
                # Actualizamos el estado de la estacion mas cercana
                nearest_station = index_m[station][nearest_station_index] # Estacion mas cercana a la nuestra
                actual_state[nearest_station] += search # Le sumamos el numero de bicicletas dejas en los slots libres
                
            tkms += travel_kms # Sumamos los kilometros recorridos al total
            
    return tkms

def find_nearest_station_index(index_m, station, actual_state, capacity, search, search_type):
    """
    Devuelve la posicion (index-columna) de la estacion mas cercana con respecto al parametro station en la variable index_m
    
    Parametros
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        station : int
            Estacion desde la que buscar su estacion vecina mas proxima
        actual_state : Numpy ndarray
            Array con el estado actual de bicicletas en cada estacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        search : int
            Numero de bicicletas o slots a buscar
        search_type : str
            Cadena de caracteres que nos indica el tipo de elemento a buscar: 'bicycles' o 'slots'
            
    Return
        Devuelve un int
    """
    # Valor que almacena la posicion (columna) de la estacion mas cercana con respecto al parametro station
    row_selec = index_m[station] # Seleccionamos el array de estaciones cercanas de la estacion station
    
    # Comprobamos que tipo de busqueda estamos realizando, 'bicycles' o 'slots'
    if search_type == 'bicycles':
        # Recorremos el array seleccionado y comprobamos cual es la estacion mas cercana con bicicletas suficientes
        for j in range(1, len(row_selec)):
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            if search <= actual_state[row_selec[j]]:
                return j # Devolvemos el index
            
    elif search_type == 'slots':
        # Recorremos el array seleccionado y comprobamos cual es la estacion mas cercana con slots suficientes
        for j in range(1, len(row_selec)):
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            free_slots = capacity[row_selec[j]] - actual_state[row_selec[j]] # Calculamos el numero de slots disponibles restandole a la capacidad total el numero de bicicletas actuales
            if search <= free_slots:
                return j # Devolvemos el index
    else:
        raise Exception("La variable search_type no contiene un tipo valido") # La palabra 'raise' nos permite lanzar una excepcion. Es igual que la palabra 'throw' en otros lenguajes
        

def show_variation(init_solution, init_tkms, new_solution, new_tkms):
    """
    Muestra por pantalla la diferencia de slots existentes entre la ambas soluciones
    
    Parametros
        init_solution : Numpy ndarray
            Solucion inicial a comparar
        new_solution : Numpy ndarray
            Solucion a comparar con respecto a init_solution
    """
    
    columns = np.arange(0,len(init_solution))
    variation = abs(init_solution - best_solution)
    
    # Para representarlo carrectamente la variacion crearemos un string con los espacios correspondientes
    string = '['
    space = ' '
    for i in range(0,len(init_solution)):
        if variation[i] < 10:
            string += space + str(variation[i])
        else:
            string += str(variation[i])
        
        if i != len(init_solution)-1:
            string += space
        
    string += ']'
    
    print('----------------------COMPARACION DE SOLUCIONES-----------------------')
    print('    Estacion    : {}'.format(columns))
    print('----------------------------------------------------------------------')
    print('Solucion inicial: {} - {} km'.format(init_solution, init_tkms))
    print(' Solucion final : {} - {} km'.format(new_solution, new_tkms))
    print('----------------------------------------------------------------------')
    print('    Variacion   : ' + string)
    print('----------------------------------------------------------------------')

### Generación de la solución inicial

**Debemos de reescribir la solucion inicial**

Crearemos un array con el tamaño de las estaciones, con valor 0. Recorreremos el array y le asignaremos un valor de slots a cada una de forma aleatoria. Para ello, tendremos un rango máximo y mínimo. De forma que, una vez calculado dicho número se hará lo mismo con la siguiente estación. Cuando se llegue a la última estación se le asignara el número restante de slots a repartir

In [1]:
def generate_initial_solution(seed, n_stations, max_slots):
    '''
    Crea una solucion a partir del numero maximo de slots disponibles
    
    Parametros
        seed : int
            Semilla con la que generar numeros aleatorios
        n_stations : int
            Numero de estaciones que tiene el problema
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    np.random.seed(seed) # Inicializamos la semilla
    solution = np.zeros(16) # Array donde guardaremos la solucion, cada valor del vector representa el numero de slots que tiene
    
    # Debemos establecer un rango de slots que podemos aniadir en cada estacion
    max_value = 30 # Numero minimo de slots que se le puede dar a una estacion
    min_value = 5 # Numero minimo de slots que se le puede dar a una estacion
    assign_slots = 0 # Numero de slots asignados en cada momento
    
    for i in range(0,n_stations):
        # Comprobamos el numero de slots que quedan por asignar
        remaining = max_slots - assign_slots
        
        # En la estacion debemos asignar todos los slots restantes
        if i != (n_stations-1):
            # Comprobamos que el valor maximo a asignar, es menor que los slots que quedan por repartir
            if remaining < max_value:
                max_value = remaining
                
            value = np.random.randint(min_value, max_value) # Creamos un numero aleatorio
            solution[i] = value
            assign_slots += value # Actualizamos el numero de slots asignados
        
        else:
            solution[i] = remaining
            
    # Ahora necesitamos mezclar las estaciones, ya que es probable que las primeras estaciones tengan mas slots que las ultimas
    np.random.shuffle(solution)

    return solution

## Desarrollamos la nueva función movement_operator

In [46]:
def movement_operator(actual_state, actual_solution, neighbours_limit, n_slots):
    """
    
    """
    
    n = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles
    neighbours_list = np.array([]) # Lista de vecinos encontrados
    neighbours_n = 0 # Numero de vecinos encontrados
    
    # Recorremos todos los posibles vecinos (movimiento de n_slots entre todas las combinaciones de estaciones posibles)
    for i in range(1, n):
        for j in range(i, n): 
            # Buscamos todos los vecinos posibles que no superen nuestro limite de exploracion
            if neighbours_n < neighbours_limit:
                # Ahora comprobamos si cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
                if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
                    neighbours_n += 1 # Aumentamos el numero de vecinos encontrados
                    
                    new_neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
                    new_neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
                    new_neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'
                    
                    if len(neighbours_list) == 0:
                        neighbours_list = np.array([new_neighbour])
                    else:
                        neighbours_list = np.append(neighbours_list, np.array([new_neighbour]), axis=0) # Aniadimos una nueva fila(axis=0) a la lista de vecinos. axis=0 para que la nueva solucion se aniada como fila
                        
    # Devolvemos el mejor vecino
    return neighbours_list

# Pruebas

In [55]:
import numpy as np
import pandas as pd

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix

seed = 1 # Semilla inicial
n_slots = 5 # Numero de slots a mover en cada movimiento
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
init_solution = generate_initial_solution(init_state, 220, seed) # Generamos la solucion inicial
move_list = movements_to_list(deltas_df) # Creamos la lista de movimientos


neighbours_limit = 120 # El numero de vecinos posibles para 16 estaciones es de 120
actual_solution = init_solution.copy()
init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos nuestra solucion
best_solution = actual_solution.copy()index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
best_tkms = init_tkms

print('Init_solution: {} - {} kms'.format(init_solution, best_tkms))

i = 0
neigbours_list = movement_operator(init_state, actual_solution, neighbours_limit, n_slots) 
print('Se han encontrado {}'.format(len(neigbour_list)))

for neighbour in neigbours_list:
    new_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m) # Evaluamos nuestra solucion
    if best_tkms > new_tkms:
        best_solution = neighbour.copy()
        best_tkms = new_tkms
        print('New best_solution: {} - {} kms '.format(best_solution, best_tkms))
    
    i += 1

print('')
show_variation(init_solution, init_tkms, best_solution, best_tkms)

Init_solution: [ 9 11 16  7 11 16 11 14  9 16 11 21 14 18 18 18] - 69.51756524864952 kms
Se han encontrado 120
New best_solution: [ 9 11 16  7 11 16 11  9  9 21 11 21 14 18 18 18] - 64.3924453097848 kms 
New best_solution: [ 9 11 16  7 11 16 11 14  9 16 11 21  9 18 18 23] - 63.20515963272674 kms 
New best_solution: [ 9 11 16  7 11 16 11 14  9 16 11 21 14 13 18 23] - 59.37142332674604 kms 

----------------------COMPARACION DE SOLUCIONES-----------------------
    Estacion    : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
----------------------------------------------------------------------
Solucion inicial: [ 9 11 16  7 11 16 11 14  9 16 11 21 14 18 18 18] - 69.51756524864952 km
 Solucion final : [ 9 11 16  7 11 16 11 14  9 16 11 21 14 13 18 23] - 59.37142332674604 km
----------------------------------------------------------------------
    Variacion   : [ 0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  5]
----------------------------------------------------------------------


In [27]:
neighbours_list = np.array([]) # Lista de vecinos encontrados
print(len(neighbours_list))

0
